In [14]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DataFrameLoader
import sentence_transformers
from langchain.vectorstores import Chroma
import pandas as pd

In [2]:
%cd /Volumes/External/source/GTPracticum/data 
%ls -salh

/Volumes/External/source/GTPracticum/data
total 13153304
      0 drwxr-xr-x   17 kariato  staff   544B Feb 21 00:04 ./
      0 drwxr-xr-x    7 kariato  staff   224B Feb 10 21:29 ../
     16 -rw-r--r--@   1 kariato  staff   6.0K Feb 10 14:58 .DS_Store
  27280 -rw-r--r--@   1 kariato  staff    13M Feb 21 00:04 2023-10-eb-big-book-of-mlops-2nd-edition-v2-102723-final (1).pdf
  27280 -rw-r--r--@   1 kariato  staff    13M Feb 20 10:15 2023-10-eb-big-book-of-mlops-2nd-edition-v2-102723-final.pdf
    464 -rw-r--r--@   1 kariato  staff   232K Feb 10 16:00 9deec473-6ab3-4c90-a2ec-86ae93b0a6e6.pdf
 733000 -rw-r--r--@   1 kariato  staff   358M Feb 10 14:56 archive (1).zip
      0 drwxr-xr-x    4 kariato  staff   128B Feb 13 22:03 chroma2_db/
      0 drwxr-xr-x    4 kariato  staff   128B Feb 12 02:29 chroma_db/
2785400 -rw-rw-r--@   1 kariato  staff   1.3G Sep 19  2019 emails.csv
2099696 -rw-r--r--    1 kariato  staff   1.0G Feb 10 14:24 enron.sql
 427936 -rw-r--r--@   1 kariato  staff   209M Feb 

In [3]:
enron_df=pd.read_csv('enron_processed.csv')

In [4]:
print(enron_df.shape)
enron_df.drop_duplicates(subset='body_short',inplace=True) 
print(enron_df.shape)


(517401, 15)
(246415, 15)


In [5]:
enron_df.drop(['body','raw'],axis=1,inplace=True)

In [6]:
enron_df.columns

Index(['Unnamed: 0', 'MESSAGE-ID', 'DATE', 'FROM', 'TO', 'SUBJECT',
       'MIME-VERSION', 'CONTENT-TYPE', 'CONTENT-TRANSFER-ENCODING',
       'body_short', 'CC', 'SENT', 'BCC'],
      dtype='object')

In [7]:
load_df = DataFrameLoader(enron_df, page_content_column='body_short')
documents = load_df.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [9]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embedding_function2 = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

/Volumes/External/opt/anaconda3/envs/ollama/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Volumes/External/opt/anaconda3/envs/ollama/lib/python3.10/site-packages/torch/_utils.py:841: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
db = Chroma.from_documents(docs, embedding_function,persist_directory="./chroma_db")

db2 = Chroma.from_documents(docs, embedding_function2,persist_directory="./chroma2_db")

In [13]:
db3 = Chroma.from_documents(docs, embedding_function2,persist_directory="./chroma3_db",collection_metadata={"hnsw:space": "cosine"})

In [9]:
db2 = Chroma.from_documents(docs, embedding_function2, persist_directory="./chroma2_db")

In [39]:
query = "illegal manipluation"
docs = db.similarity_search(query)

# print results
print(docs[2])

page_content="Subject: Re: Socal\n\nCorporate schizophrenia.  Weird.   DF\n\n\n   \n\t\n\t\n\t\n\n\nSubject: Socal\n\nSoCal's protest is ludicrous. I will work on a response.\nBy the way, SoCal has actually been remarkably helpful in reaching agreement \non a settlement -- which we just signed -- in the California GIR proceeding." metadata={'DATE': ' Mon, 17 Apr 2000 09:52', 'FROM': '>>   Susan Scott                           04/17/2000 03', 'SUBJECT': ' Re>> ', 'TO': '>> ', 'Unnamed: 0': 115182}


In [13]:
#  'similarity_search',
#  'similarity_search_by_vector',
#  'similarity_search_by_vector_with_relevance_scores',
#  'similarity_search_with_relevance_scores',
#  'similarity_search_with_score'
query = "illegal manipluation"
docs = db2.similarity_search_with_score(query)
print(docs[0])

(Document(page_content='Subject: Re: Copano P/L 01/00 - S93481\n\nCynthia/Daren:\n\nKim Vaughn has taken over all of Kathy\'s meters.  I will look at this issue \nwith Kim since she is "new".\n\nFred\n\n\n   \n\t\n\t\n\t\n\nSubject: Re: Copano P/L 01/00 - S93481\n\nFred, Can you help with this since Kathy isn\'t here anymore?\n---------------------- Forwarded by Cynthia Hakemack/HOU/ECT on 01/08/2001 \n\n\nDaren J Farmer\n01/08/2001 03:54 PM\nSubject: Re: Copano P/L 01/00 - S93481  \n\nThis is a call out meter.  I think the best person to help you would be Kathy \nBenedict in Volume Management.\n\nD\n\n\n   \n\t\n\t\n\t\n\nSubject: Copano P/L 01/00 - S93481\n\nDaren, I\'m looking at S93481 for 01/00.  3,461 mmbtu was scheduled to Copano \nP/L in 02/00 and the counterparty was paid.  In 05/00 the volumes were \nreversed from this deal.  Would you be able to tell why they were reversed?  \nI\'m trying to determine if this is a valid recoverable item.  Thanks.', metadata={'CC': '>> >> >> 

In [15]:
#  'similarity_search',
#  'similarity_search_by_vector',
#  'similarity_search_by_vector_with_relevance_scores',
#  'similarity_search_with_relevance_scores',
#  'similarity_search_with_score'
query = "illegal manipluation"
docs = db2.similarity_search_with_relevance_scores(query)
print(docs[0])

(Document(page_content='Subject: Re: Copano P/L 01/00 - S93481\n\nCynthia/Daren:\n\nKim Vaughn has taken over all of Kathy\'s meters.  I will look at this issue \nwith Kim since she is "new".\n\nFred\n\n\n   \n\t\n\t\n\t\n\nSubject: Re: Copano P/L 01/00 - S93481\n\nFred, Can you help with this since Kathy isn\'t here anymore?\n---------------------- Forwarded by Cynthia Hakemack/HOU/ECT on 01/08/2001 \n\n\nDaren J Farmer\n01/08/2001 03:54 PM\nSubject: Re: Copano P/L 01/00 - S93481  \n\nThis is a call out meter.  I think the best person to help you would be Kathy \nBenedict in Volume Management.\n\nD\n\n\n   \n\t\n\t\n\t\n\nSubject: Copano P/L 01/00 - S93481\n\nDaren, I\'m looking at S93481 for 01/00.  3,461 mmbtu was scheduled to Copano \nP/L in 02/00 and the counterparty was paid.  In 05/00 the volumes were \nreversed from this deal.  Would you be able to tell why they were reversed?  \nI\'m trying to determine if this is a valid recoverable item.  Thanks.', metadata={'CC': '>> >> >> 

: 

In [40]:
print(enron_df[enron_df['Unnamed: 0']==115182]['raw'].to_list()[0])

Message-ID: <12645594.1075842517758.JavaMail.evans@thyme>
Date: Mon, 17 Apr 2000 09:52:00 -0700 (PDT)
From: drew.fossum@enron.com
To: susan.scott@enron.com
Subject: Re: Socal
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Drew Fossum
X-To: Susan Scott
X-cc: 
X-bcc: 
X-Folder: \Drew_Fossum_Dec2000_June2001_1\Notes Folders\'sent mail
X-Origin: FOSSUM-D
X-FileName: dfossum.nsf

Corporate schizophrenia.  Weird.   DF


   
	
	
	From:  Susan Scott                           04/17/2000 03:17 PM
	

To: Drew Fossum@ENRON
cc:  

Subject: Socal

SoCal's protest is ludicrous. I will work on a response.
By the way, SoCal has actually been remarkably helpful in reaching agreement 
on a settlement -- which we just signed -- in the California GIR proceeding.




In [28]:
print(enron_df[enron_df['Unnamed: 0']==32653]['body'].to_list()[0])

Subject: Energy market outlook 11/27/01

